In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai
!pip install summac

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 22.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 12.7 MB/s eta 0:00:00
 

In [ ]:
%cd drive/Shareddrives/'CSCI 544 Projects'/'Yelp ACOS'

/content/drive/Shareddrives/CSCI 544 Projects/Yelp ACOS


In [ ]:
import random
import json
import openai
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from summac.model_summac import SummaCZS
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
acos_pred_only = "yelp-acos-_ab50qdWOk0DdB6XOrBitw-200.jsonl"
acos_pred_category = "yelp-acos-_ab50qdWOk0DdB6XOrBitw-200-category-groups.json"
acos_pred_aspect = "yelp-acos-_ab50qdWOk0DdB6XOrBitw-200-aspect-groups.json"

review_sample = "yelp-acos-100-sampled-rest-reviews.json"

### Read in quads from JSON

In [ ]:
# Read in 200 reviews each with ACOS predictions - Each line is a JSON dictionary
reviews_acos = []
with open(acos_pred_only, "r") as f:
    for line in f:
        review = json.loads(line)
        reviews_acos.append(review)

In [ ]:
# Read in reviews grouped by category, 5 category groups were chosen
# len(reviews_acos_category) = 5
# each category group has three items: group_name, group_size, group
reviews_acos_category = {}
with open(acos_pred_category, "r") as f:
    reviews_acos_category = json.load(f)

In [ ]:
# 5 Categories
print("Sorted by categories")
categories = []
for i in range(0, len(reviews_acos_category)):
    categories.append(reviews_acos_category[i]['group_name'])
    
for i in range(0, len(reviews_acos_category)):
    print(reviews_acos_category[i]['group_name'] + ":", reviews_acos_category[i]['group_size'], "reviews")

Sorted by categories
QUALITY: 532 reviews
GENERAL: 244 reviews
STYLE_OPTIONS: 113 reviews
SERVICE: 52 reviews
PRICES: 22 reviews


In [ ]:
# Read in reviews grouped by aspect, 7 aspects were chosen
# len(reviews_acos_aspect) = 7
# each aspect has three items: group_name, group_size, group
reviews_acos_aspect = {}
with open(acos_pred_aspect, "r") as f:
    reviews_acos_aspect = json.load(f)

In [ ]:
# 7 Aspects
print("Sorted by aspects")
aspects = []
for i in range(0, len(reviews_acos_aspect)):
    aspects.append(reviews_acos_aspect[i]['group_name'])

for i in range(0, len(reviews_acos_aspect)):
    print(reviews_acos_aspect[i]['group_name'] + ":", reviews_acos_aspect[i]['group_size'], "reviews")

Sorted by aspects
oysters: 54 reviews
service: 45 reviews
chargrilled oysters: 43 reviews
food: 41 reviews
Acme: 29 reviews
place: 24 reviews
gumbo: 22 reviews


### Use GPT to turn these quads into summary about that specific aspect for 1 restaurant
#### Pass just quads (Grouped by Categories) to gpt/davinci to generate summary

In [ ]:
categories_output = []

In [ ]:
# Compile all acos quadruples for each category/aspect
# returns a list of acos quadruples for said aspect
def compile_acos_review(group_index, review_acos_dictionary):
    # dict of review to acos quads for said group
    review2acos = {}
    group_name = review_acos_dictionary[group_index]['group_name']
    for i in range(review_acos_dictionary[group_index]['group_size']): 
        review = review_acos_dictionary[group_index]['group'][i]['review']
        # duplication of reviews
        if review in review2acos:
            continue
        # all acos quads of review
        acos_quads = review_acos_dictionary[group_index]['group'][i]['acos_preds']
        # acos quads with group name
        acos_quads_group = [x for x in acos_quads if x['category'] == group_name]
        review2acos[review] = acos_quads_group
    return review2acos

# Function to generate a prompt
def generate_prompt(group_name, review2acos, max_len):
    
    # Format the acos quadruples into a string
    def acos2str(review2acos):
        formatted_acos = ""
        for review, acos_quads in review2acos.items():
            formatted_acos += review + " <ACOS> "
            for i in range(len(acos_quads)):
                quad = acos_quads[i]
                aspect = ' '.join(quad['aspect']) if isinstance(quad['aspect'], (list, tuple)) else quad['aspect']
                opinion = ' '.join(quad['opinion']) if isinstance(quad['opinion'], (list, tuple)) else quad['opinion']
                #category = item['category']
                sentiment = quad['sentiment']
                #formatted_acos += f"{i+1}. Aspect: {aspect}, Category: {category}, Opinion: {opinion}, Sentiment: {sentiment}\n"
                formatted_acos += f"{i+1}. Aspect: {aspect}, Opinion: {opinion}, Sentiment: {sentiment} "
            formatted_acos += " </ACOS> \n"
        return formatted_acos
    
    # choose the first max_len reviews
    review2acos = dict(list(review2acos.items())[:max_len])
    # randomly sample max_len reviews
    # random.seed(0)
    # review2acos = dict(random.sample(review2acos.items(), max_len))
    prompt = f'Summarize the following restaurant reviews with at most 5 sentences. The reviews are followed by their corresponding aspect-category-opinion-sentiment quadruples enclosed between <ACOS> and </ACOS>. The summarization should only be about "{group_name}". '
    formatted_acos_string = acos2str(review2acos)
    
    prompt = prompt + formatted_acos_string
    
    return prompt

def generate_summary_category(group_index):
    review2acos = compile_acos_review(group_index, reviews_acos_category)

    openai_model = "text-davinci-003"
    group_name = categories[group_index]

    prompt = generate_prompt(group_name, review2acos, 10)

    resp = openai.Completion.create(
        model = openai_model,
        prompt = prompt,
        temperature = 0,
        max_tokens = 1000
    )

    summary_str = resp['choices'][0]['text'].replace('\n', '')
    temp_tuple = (group_name, summary_str)
    
    return temp_tuple

In [ ]:
for i in range(len(reviews_acos_category)):
    temp_tuple = generate_summary_category(i)
    categories_output.append(temp_tuple)

In [ ]:
for category, text in categories_output:
    print(f"{category}:\n{text}\n")

QUALITY:
Summarizing the reviews, Acme offers amazing quality oysters, with the chargrilled oysters being particularly phenomenal. The bloody mary is also highly praised, as well as the seafood gumbo and the piece po-boy. The service is also top notch.

GENERAL:
Acme is a popular seafood restaurant in New Orleans with great service and oysters. It is worth the wait to get in, and customers recommend trying the char-grilled oysters. Other restaurants in the area are also recommended for fried seafood.

STYLE_OPTIONS:
Summarizing the reviews about "STYLE_OPTIONS", Acme Oyster House offers a variety of fried seafood options, including chargrilled oysters, 1/2 and 1/2 po'boy, fried crawfish tail appetizer, and a seafood platter. The atmosphere is casual and there is no need to dress up or make a reservation. However, there is usually a long line to get in and the shucking of the oysters leaves all the sandy mud on the outside of the shell.

SERVICE:
Overall, reviews of Acme Oyster House's 

#### Pass just quads (Grouped by Aspect) to gpt/davinci to generate summary

In [ ]:
aspect_output = []

In [ ]:
# Compile all acos quadruples for each category/aspect
# returns a list of acos quadruples for said aspect
def compile_acos_review(group_index, review_acos_dictionary):
    review2acos = {}
    group_name = review_acos_dictionary[group_index]['group_name']
    #print(group_name)
    for i in range(review_acos_dictionary[group_index]['group_size']):      
        review = review_acos_dictionary[group_index]['group'][i]['review']  
        if review in review2acos:
            continue
        acos_quads = review_acos_dictionary[group_index]['group'][i]['acos_preds']
        acos_quads_group = [x for x in acos_quads if x['aspect'] is not None and group_name in x['aspect']]
        review2acos[review] = acos_quads_group
    return review2acos

# Function to generate a prompt
def generate_prompt(group_name, review2acos, max_len):
    
    # Format the acos quadruples into a string
    def acos2str(review2acos):
        formatted_acos = ""
        for review, acos_quads in review2acos.items():
            formatted_acos += review + " <ACOS> "
            for i in range(len(acos_quads)):
                quad = acos_quads[i]
                # aspect = ' '.join(quad['aspect']) if isinstance(quad['aspect'], (list, tuple)) else quad['aspect']
                opinion = ' '.join(quad['opinion']) if isinstance(quad['opinion'], (list, tuple)) else quad['opinion']
                category = quad['category']
                sentiment = quad['sentiment']
                #formatted_acos += f"{i+1}. Aspect: {aspect}, Category: {category}, Opinion: {opinion}, Sentiment: {sentiment}\n"
                formatted_acos += f"{i+1}. Category: {category}, Opinion: {opinion}, Sentiment: {sentiment} "
            formatted_acos += " </ACOS> \n"
        return formatted_acos
    
    # choose the first max_len reviews
    review2acos = dict(list(review2acos.items())[:max_len])
    # prompt = "Based on the following restaurant reviews along with their corresponding aspect-category-opinion-sentiment (ACOS) quadruples enclosed between <ACOS> and </ACOS>, please only provide a summary about the " + group_name + " of the restaurant with at most 5 sentences. Don't include any ACOS quadruples in your answer: "
    prompt = f'Summarize the following restaurant reviews with at most 5 sentences. The reviews are followed by their corresponding aspect-category-opinion-sentiment quadruples enclosed between <ACOS> and </ACOS>. The summarization should only be about "{group_name}". '
    formatted_acos_string = acos2str(review2acos)
        
    prompt = prompt + formatted_acos_string
    
    return prompt

def generate_summary_aspects(group_index):
    acos_quads_quality = compile_acos_review(group_index, reviews_acos_aspect)

    openai_model = "text-davinci-003"
    group_name = aspects[group_index]

    prompt = generate_prompt(group_name, acos_quads_quality, 10)

    resp = openai.Completion.create(
        model = openai_model,
        prompt = prompt,
        temperature = 0,
        max_tokens = 1000
    )

    summary_str = resp['choices'][0]['text'].replace('\n', '')
    temp_tuple = (group_name, summary_str)
    
    return temp_tuple

In [ ]:
for i in range(len(reviews_acos_aspect)):
    temp_tuple = generate_summary_aspects(i)
    aspect_output.append(temp_tuple)

In [ ]:
for aspect, text in aspect_output:
    print(f"{aspect}:\n{text}\n")

oysters:
Summarizing reviews about oysters, customers have praised the quality of the oysters, describing them as amazing, phenomenal, salty sweet, and great. They have also recommended getting there early as the wait could be long.

service:
Overall, reviews of the restaurant's service were positive, with customers praising the friendly staff and good service.

chargrilled oysters:
Acme Oyster House is a must-visit spot in New Orleans for chargrilled oysters. Many reviews have praised the chargrilled oysters for their unique flavor, with garlic, parmesan cheese, and butter. Other dishes such as the 1/2 and 1/2 po'boy, fried crawfish tail appetizer, seafood gumbo, and oyster poboy are also available. The atmosphere is casual and the service is decent. However, it is best to visit off-peak to avoid long queues.

food:
The restaurant reviews indicate that the food is of great quality, with many praising the freshness and flavor of the oysters, as well as the fried and chargrilled dishes.

### Evaluation

In [ ]:
reviews = [x['review'] for x in reviews_acos]

In [ ]:
model_zs = SummaCZS(granularity="sentence", model_name="vitc", device="cuda") # If you have a GPU: switch to: device="cuda"

In [ ]:
# convert review samples to list of strings
with open(review_sample, "r") as f:
    reviews_rest = json.load(f)
reviews_rest_concat = []
for restaurant in reviews_rest: 
    revs = ""
    for rev in restaurant['reviews']:
        revs += rev
    reviews_rest_concat.append(revs)

In [ ]:
def split_and_rephrase(sum):
    sum1 = "The restaurant reviews cover a range of opinions on food quality, with many customers raving about the food. Several reviews praise the presentation of the food, and others mention specific dishes that were particularly good. There are also some negative reviews, with complaints about inconsistent quality, downsized portions, high prices, and poor service. However, overall, the majority of reviews are positive, with many customers recommending the restaurant and its food. The reviews cover a range of cuisines, including Mexican, Indian, and sushi, with some specific dishes mentioned as standouts."
    sum1_rephrased = "The presentation of the food is praised .\n Food quality is inconsistent .\n Portions are downsized .\n Prices are high .\n Service is poor .\n The restaurant and its food are recommended .\n Mexican food is good .\n Indian food is good .\n Sushi is good ."
    sum2 = "The restaurant reviews cover a variety of food types and service experiences. Some of the positive reviews include comments about the best tuna ever had, great value sushi with high quality, superb caesar salad, and the best crab cakes in town. On the other hand, some of the negative reviews mention poor customer service, poor quality pizza, and overpriced food. Additionally, some reviewers mention specific dishes they enjoyed such as the seabass on lobster risotto, honey walnut prawns, asparagus, and lobster 3 ways. Overall, there are mixed reviews, with some praising the food quality while others criticize the service and value."
    sum2_rephrased = "The tuna is the best that customers ever had .\n Sushi is of great value and high quality .\n Caesar salad is superb .\n Crab cakes are the best in town .\n Customer service is poor .\n Pizza is of poor quality .\n Food is overpriced .\n Customers enjoyed the seabass on lobster risotto .\n Customers enjoyed honey walnut prawns .\n Customers enjoyed asparagus .\n Customers enjoyed lobster 3 ways ."
    prompt = "Split and rephrase the following sentences into simple propositions: "
    input = prompt + sum1 + "\n Output: " + sum1_rephrased + "\n" + prompt + sum2 + "\n Output: " + sum2_rephrased + "\n" + prompt + sum + "\n Output: "
    # print(prompt)
    resp = openai.Completion.create(
        model="text-davinci-003",
        prompt=input,
        temperature=0,
        max_tokens=1000
    )
    pred_str = resp['choices'][0]['text']
    return pred_str

In [ ]:
def compute_faithfulness(reviews, summary, t):
    avg_support = 0
    for sum in summary:
        support = 0
        # reviews_batch = random.sample(reviews, 20)
        for review in reviews:
            score_zs = model_zs.score([review], [sum])['scores'][0]
            # if score_zs > 0.2:
            if score_zs > t:
                support += 1
        avg_support += support

        print("summary:", sum)
        print("support:", support)
        print("=====")

    avg_support /= len(summary)
    print("faithfulness:", avg_support)
    return avg_support

In [ ]:
def compute_factuality(reviews, summary):
    avg_topscore = 0
    for sum in summary:
        top_score = 0
        # reviews_batch = random.sample(reviews, 20)
        for review in reviews:
            score_zs = model_zs.score([review], [sum])['scores'][0]
            top_score = max(top_score, score_zs)
        avg_topscore += top_score

        print("summary:", sum)
        print("top score:", top_score)
        print("=====")

    avg_topscore /= len(summary)
    print("factuality:", avg_topscore)
    return avg_topscore

In [ ]:
def compute_genericity(reviews, summary):
    # remove stop words and puctuations
    stop_words = set(stopwords.words('english'))
    summary_words = []
    for sum in summary:
        sum = sum.split(" ")
        sum = [word.lower() for word in sum if word not in stop_words and len(word) > 0 and word != "."]
        summary_words += sum

    avg_idf = 0
    for word in summary_words:
        count = 0
        for review in reviews:
            count += (review.find(word) != -1)
        # plus-one smoothing in case count = 0
        idf = len(reviews) / (count+1)
        avg_idf += idf

        print("word in summary:", word)
        print("count:", count)
        print("=====")

    avg_idf /= len(summary_words)
    print("genericity:", avg_idf)
    return avg_idf

In [ ]:
# summaries grouped by category
summary_file = 'yelp_summary_category_review+acos'
with open(summary_file, 'r') as f:
    summary = f.read()
summary_split = split_and_rephrase(summary).split('\n')
summary_split = [x for x in summary_split if len(x) > 0]

In [ ]:
compute_faithfulness(reviews, summary_split, 0.2)


summary:  Oysters are of amazing quality .
support: 47
=====
summary:  Chargrilled oysters are phenomenal .
support: 46
=====
summary:  Bloody mary is highly praised .
support: 14
=====
summary:  Seafood gumbo is highly praised .
support: 11
=====
summary:  Po-boy is highly praised .
support: 33
=====
summary:  Service is top notch .
support: 52
=====
summary:  Acme is a popular seafood restaurant .
support: 6
=====
summary:  It is worth the wait to get in .
support: 28
=====
summary:  Customers recommend trying the char-grilled oysters .
support: 16
=====
summary:  Other restaurants in the area are recommended for fried seafood .
support: 0
=====
summary:  Acme offers a variety of fried seafood options .
support: 25
=====
summary:  Atmosphere is casual .
support: 7
=====
summary:  No need to dress up or make a reservation .
support: 1
=====
summary:  Usually a long line to get in .
support: 45
=====
summary:  Shucking of the oysters leaves sandy mud on the shell .
support: 1
=====
su

23.047619047619047

In [ ]:
compute_factuality(reviews, summary_split)

summary:  Oysters are of amazing quality .
top score: 0.925384521484375
=====
summary:  Chargrilled oysters are phenomenal .
top score: 0.9068145751953125
=====
summary:  Bloody mary is highly praised .
top score: 0.715057373046875
=====
summary:  Seafood gumbo is highly praised .
top score: 0.674163818359375
=====
summary:  Po-boy is highly praised .
top score: 0.8076019287109375
=====
summary:  Service is top notch .
top score: 0.9620819091796875
=====
summary:  Acme is a popular seafood restaurant .
top score: 0.59210205078125
=====
summary:  It is worth the wait to get in .
top score: 0.960235595703125
=====
summary:  Customers recommend trying the char-grilled oysters .
top score: 0.83477783203125
=====
summary:  Other restaurants in the area are recommended for fried seafood .
top score: 0.1583251953125
=====
summary:  Acme offers a variety of fried seafood options .
top score: 0.77606201171875
=====
summary:  Atmosphere is casual .
top score: 0.849090576171875
=====
summary:  No

0.7660616920107887

In [ ]:
compute_genericity(reviews_rest_concat, summary_split)

word in summary: oysters
count: 8
=====
word in summary: amazing
count: 58
=====
word in summary: quality
count: 45
=====
word in summary: chargrilled
count: 0
=====
word in summary: oysters
count: 8
=====
word in summary: phenomenal
count: 6
=====
word in summary: bloody
count: 2
=====
word in summary: mary
count: 1
=====
word in summary: highly
count: 22
=====
word in summary: praised
count: 0
=====
word in summary: seafood
count: 16
=====
word in summary: gumbo
count: 2
=====
word in summary: highly
count: 22
=====
word in summary: praised
count: 0
=====
word in summary: po-boy
count: 4
=====
word in summary: highly
count: 22
=====
word in summary: praised
count: 0
=====
word in summary: service
count: 91
=====
word in summary: top
count: 73
=====
word in summary: notch
count: 13
=====
word in summary: acme
count: 0
=====
word in summary: popular
count: 5
=====
word in summary: seafood
count: 16
=====
word in summary: restaurant
count: 73
=====
word in summary: it
count: 100
=====
w

19.63903818741564

In [ ]:
# summaries grouped by aspect
summary_file = 'yelp_summary_aspect_review+acos'
with open(summary_file, 'r') as f:
    summary = f.read()
summary_split = split_and_rephrase(summary).split('\n')
summary_split = [x for x in summary_split if len(x) > 0]

In [ ]:
summary_split

['The oysters are amazing .',
 'The oysters are phenomenal .',
 'The oysters are salty sweet .',
 'The oysters are great .',
 'Get there early to avoid long queues .',
 'The staff is friendly .',
 'The service is good .',
 'Garlic, parmesan cheese, and butter are used for chargrilled oysters .',
 "The 1/2 and 1/2 po'boy is available .",
 'The fried crawfish tail appetizer is available .',
 'Seafood gumbo is available .',
 'Oyster poboy is available .',
 'The atmosphere is casual .',
 'The food is of great quality .',
 'The food is fresh .',
 'The food is flavorful .',
 'The fried and chargrilled dishes are praised .',
 'Acme is highly recommended .',
 'The charbroiled oysters are particularly praised .',
 'The place is popular .',
 'The food is delicious .',
 'The staff is friendly and knowledgeable .',
 'Gumbo is great .',
 'Gumbo is delicious .',
 'Gumbo is very tasty .',
 'Gumbo is just OK.']

In [ ]:
compute_faithfulness(reviews, summary_split, 0.2)

summary: The oysters are amazing .
support: 64
=====
summary: The oysters are phenomenal .
support: 59
=====
summary: The oysters are salty sweet .
support: 3
=====
summary: The oysters are great .
support: 78
=====
summary: Get there early to avoid long queues .
support: 9
=====
summary: The staff is friendly .
support: 32
=====
summary: The service is good .
support: 78
=====
summary: Garlic, parmesan cheese, and butter are used for chargrilled oysters .
support: 0
=====
summary: The 1/2 and 1/2 po'boy is available .
support: 1
=====
summary: The fried crawfish tail appetizer is available .
support: 3
=====
summary: Seafood gumbo is available .
support: 23
=====
summary: Oyster poboy is available .
support: 15
=====
summary: The atmosphere is casual .
support: 7
=====
summary: The food is of great quality .
support: 80
=====
summary: The food is fresh .
support: 17
=====
summary: The food is flavorful .
support: 92
=====
summary: The fried and chargrilled dishes are praised .
support

32.30769230769231

In [ ]:
compute_factuality(reviews, summary_split)

summary: The oysters are amazing .
top score: 0.954620361328125
=====
summary: The oysters are phenomenal .
top score: 0.923187255859375
=====
summary: The oysters are salty sweet .
top score: 0.50732421875
=====
summary: The oysters are great .
top score: 0.953125
=====
summary: Get there early to avoid long queues .
top score: 0.955322265625
=====
summary: The staff is friendly .
top score: 0.9826583862304688
=====
summary: The service is good .
top score: 0.9617156982421875
=====
summary: Garlic, parmesan cheese, and butter are used for chargrilled oysters .
top score: 0
=====
summary: The 1/2 and 1/2 po'boy is available .
top score: 0.229248046875
=====
summary: The fried crawfish tail appetizer is available .
top score: 0.4822998046875
=====
summary: Seafood gumbo is available .
top score: 0.918487548828125
=====
summary: Oyster poboy is available .
top score: 0.81842041015625
=====
summary: The atmosphere is casual .
top score: 0.867767333984375
=====
summary: The food is of grea

0.7803131983830378

In [ ]:
compute_genericity(reviews_rest_concat, summary_split)

word in summary: the
count: 100
=====
word in summary: oysters
count: 8
=====
word in summary: amazing
count: 58
=====
word in summary: the
count: 100
=====
word in summary: oysters
count: 8
=====
word in summary: phenomenal
count: 6
=====
word in summary: the
count: 100
=====
word in summary: oysters
count: 8
=====
word in summary: salty
count: 7
=====
word in summary: sweet
count: 38
=====
word in summary: the
count: 100
=====
word in summary: oysters
count: 8
=====
word in summary: great
count: 91
=====
word in summary: get
count: 93
=====
word in summary: early
count: 30
=====
word in summary: avoid
count: 10
=====
word in summary: long
count: 58
=====
word in summary: queues
count: 0
=====
word in summary: the
count: 100
=====
word in summary: staff
count: 76
=====
word in summary: friendly
count: 78
=====
word in summary: the
count: 100
=====
word in summary: service
count: 91
=====
word in summary: good
count: 99
=====
word in summary: garlic,
count: 4
=====
word in summary: par

14.654040187428482